In [115]:
import sys
import re
import numpy as np
import pandas as pd

import pygal

from pygal_maps_fr import maps
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# requires installing
# https://gtk-win.sourceforge.io/home/index.php/Main/Downloads


In [116]:
filepath = r"C:\Users\to202835\OneDrive - ATR\_exploitation\formation\2022\Udacity_DataScience\01_Datascience\project_01\workspace_1\project_01_main_v1.2.0\table-indicateurs-open-data-dep-2022-12-19-19h01.csv"
df = pd.read_csv(filepath.replace('\\','/'), dtype={"date": str, "dep": str, 'lib_dep': str, 'lib_reg': str})

In [117]:
print('df keys:', df.columns)

df keys: Index(['dep', 'date', 'reg', 'lib_dep', 'lib_reg', 'tx_pos', 'tx_incid', 'TO',
       'R', 'hosp', 'rea', 'rad', 'dchosp', 'reg_rea', 'incid_hosp',
       'incid_rea', 'incid_rad', 'incid_dchosp', 'reg_incid_rea', 'pos',
       'pos_7j', 'cv_dose1'],
      dtype='object')


In [118]:
df.head(4)

,dep,date,reg,lib_dep,lib_reg,tx_pos,tx_incid,TO,R,hosp,rea,rad,dchosp,reg_rea,incid_hosp,incid_rea,incid_rad,incid_dchosp,reg_incid_rea,pos,pos_7j,cv_dose1
0,01,2020-03-18,84,Ain,Auvergne et Rhône-Alpes,NaN,NaN,0.062612,NaN,2,0,1,0,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,2020-03-19,84,Ain,Auvergne et Rhône-Alpes,NaN,NaN,0.132379,NaN,2,0,1,0,79,1.0,0.0,0.0,0.0,44.0,NaN,NaN,NaN
2,01,2020-03-20,84,Ain,Auvergne et Rhône-Alpes,NaN,NaN,0.155635,NaN,2,0,1,0,87,0.0,0.0,1.0,0.0,16.0,NaN,NaN,NaN
3,01,2020-03-21,84,Ain,Auvergne et Rhône-Alpes,NaN,NaN,0.173524,NaN,4,0,1,0,88,3.0,0.0,0.0,0.0,15.0,NaN,NaN,NaN


In [119]:
def display_columns_info(df):

    max_len = 0
    cols = np.array(df.columns)
    for c in cols:
        if len(c) > max_len:
            max_len = len(c)
    size = max_len + 2            
    dico = dict()
    print("column label\tnb of unique\trate of unique\t  type\t\tnb of nan\trate of nan")
    for col in cols:
        values_uniq = np.array(pd.unique(df[col]))
        nan_rate = 100 * df[col].isna().sum() / df[col].shape[0]
        print('{}{}\t{}\t\t{} %\t\t  {}  \t\t{}\t{} %'.format(col, ' '*(size - len(col)), len(values_uniq), round(100*len(values_uniq)/df[col].shape[0], 2), df[col].dtypes, df[col].isna().sum(), round(nan_rate,3)))
        if col == 'date':
            print('analyse dates')
            for value in values_uniq:
                if not isinstance(value, str):
                    print('\t', value)
                else:
                    if len(value) != 10:
                        print('\t', value)
        dico[col] = values_uniq

display_columns_info(df)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
dep            	101		0.1 %		  object  		0	0.0 %
date           	1007		0.99 %		  object  		0	0.0 %
analyse dates
reg            	18		0.02 %		  int64  		0	0.0 %
lib_dep        	101		0.1 %		  object  		0	0.0 %
lib_reg        	18		0.02 %		  object  		0	0.0 %
tx_pos         	4582		4.51 %		  float64  		5959	5.859 %
tx_incid       	50498		49.65 %		  float64  		5959	5.859 %
TO             	3779		3.72 %		  float64  		0	0.0 %
R              	188		0.18 %		  float64  		86460	85.009 %
hosp           	1672		1.64 %		  int64  		0	0.0 %
rea            	457		0.45 %		  int64  		0	0.0 %
rad            	16277		16.0 %		  int64  		0	0.0 %
dchosp         	4927		4.84 %		  int64  		0	0.0 %
reg_rea        	948		0.93 %		  int64  		0	0.0 %
incid_hosp     	244		0.24 %		  float64  		101	0.099 %
incid_rea      	71		0.07 %		  float64  		101	0.099 %
incid_rad      	175		0.17 %		  float64  		101	0.099 %
incid_dchosp   	61		0.06 %		  float64  		101	0.

In [120]:
# Convert id of dep and reg to string (some are in str and some are in int)
df['reg'] = df['reg'].astype(str)
df['dep'] = df['dep'].astype(str)
df['date'] = df['date'].astype(str)
df['lib_reg'] = df['lib_reg'].astype(str)
df['lib_dep'] = df['lib_dep'].astype(str)
print('reg', df['reg'].dtypes)
print('dep', df['dep'].dtypes)
print('date', df['date'].dtypes)
print('lib_reg', df['lib_reg'].dtypes)
print('lib_dep', df['lib_dep'].dtypes)
# it does not seem to have the expected effect!

reg object
dep object
date object
lib_reg object
lib_dep object


In [121]:
def get_cumul_days():
    '''
    DESCRIPTION
        Return a list to ease count, month per month, passed day from the
         beginning of the year.
    INPUT
        nil
    OUTPUT
        cnt_day_at_start_month is a list that gives the nb of past days from
         beginning of the year for every month
    '''
    cnt_day_at_start_month = []

    day_by_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    cnt = 0

    for i in day_by_month:
        cnt += i 
        cnt_day_at_start_month.append(cnt)

    return cnt_day_at_start_month
get_cumul_days()

[0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]

In [122]:
def get_cnt_day_at_start_month(m, calendar):
    '''
    DESCRIPTION
        get the number of past days from the beginning of year and at 
         beginning of the selected month.
    INPUT
        m is the selected month (str of int, int or float)
        calendar is a list that gives the nb of past days from beginning of the
                 year for every month
    OUTPUT
        cnt is the count of days (int) from beginning of the year at start of
         the selected month.
    '''
    cnt = calendar[int(m)-1]
    return cnt
get_cnt_day_at_start_month(8, get_cumul_days())

212

In [123]:
def get_area_names(df, aera_id_label:str, aera_name_label: str):
    '''
    DESCRIPTION
        build a dictionary of a type or area (department or region) allowing getting a name of this area from its id
    INPUT
        df is the globale dataframe
        aera_id_label is the id (str) of the area: 'dep' (department) or 'reg' (region)
        area_name_label is the name (str) of the area: 'lib_dep' (department) or 'lib_reg' (region)
    OUTPUT
        dict_areas is a dictionary: id is the key, name is the value
    '''

    dict_areas = dict()
    # get the list of area id and related names
    area_ids = np.array(df[aera_id_label])
    area_names = np.array(df[aera_name_label])
    area_id_uniq = []
    for i, area_id in enumerate(area_ids):
        if area_id not in area_id_uniq:
            area_id_uniq.append(area_id)
            dict_areas[area_id] = area_names[i]
        else:
            pass
    del df, aera_id_label, aera_name_label, area_ids, area_names, area_id_uniq
    return dict_areas


# Get a dictionary of lib_dep from dep
dict_lib_dep = get_area_names(df, 'dep', 'lib_dep')
# print('dico dept:\n', dict_lib_dep)

dict_lib_reg = get_area_names(df, 'reg', 'lib_reg')
# print('dico region:\n', dict_lib_reg)


In [124]:
def get_dep_name(dep_id):
    '''
    DESCRIPTION
        give the name of a department in accordance with its id
    INPUT
        dep_id is the id (str, int or float) of the department from which we want the name
    OUTPUT
        dep_name is the name (str) of the department related to the given id
    '''
    if type(dep_id) != 'str':
        if type(dep_id) == float:
            dep_id = int(round(dep_id, 0))
        dep_id = str(dep_id)
        if len(dep_id) == 1:
            dep_id = '0' + dep_id
    try:
        name = dict_lib_dep[dep_id]
    except:
        name = 'unknown'
    return name

t_str = get_dep_name('45')
print(t_str)
t_int = get_dep_name(21)
print(t_int)
t_short_int = get_dep_name(1)
print(t_short_int)
t_short_int = get_dep_name(float(3.0))
print(t_short_int)


Loiret
Côte-d'Or
Ain
Allier


In [125]:
def get_reg_name(reg_id):
    '''
    DESCRIPTION
        give the name of a region in accordance with its id
    INPUT
        reg_id is the id (str, int or float) of the region from which we want the name
    OUTPUT
        reg_name is the name (str) of the region related to the given id
    '''
    if type(reg_id) != 'str':
        if type(reg_id) == float:
            reg_id = int(round(reg_id, 0))
        reg_id = str(reg_id)
        # if len(reg_id) == 1:
        #     reg_id = '0' + reg_id
    try:
        name = dict_lib_reg[reg_id]
    except:
        name = 'unknown'
    return name

t_str = get_reg_name('93')
print(t_str, "vs Provence-Alpes-Côte d'Azur")
t_int = get_reg_name(93)
print(t_int, "vs Provence-Alpes-Côte d'Azur")
t_short_int = get_reg_name(4)
print(t_short_int, "vs Réunion")
t_short_int = get_reg_name(float(27.04))
print(t_short_int, "vs Bourgogne et Franche-Comté")

Provence-Alpes-Côte d'Azur vs Provence-Alpes-Côte d'Azur
Provence-Alpes-Côte d'Azur vs Provence-Alpes-Côte d'Azur
Réunion vs Réunion
Bourgogne et Franche-Comté vs Bourgogne et Franche-Comté


In [126]:
def get_dep_names(dep_ids: list):
    '''
    DESCRIPTION
        give a list of department names according to a list of department ids
    INPUT
        dep_ids is the list of department id
    OUTPUT
        dep_names is the list of department names
    '''
    dep_names = list()
    for dep_id in dep_ids:
        dep_names.append(get_dep_name(dep_id))
    return dep_names

def get_reg_names(reg_ids: list):
    '''
    DESCRIPTION
        give a list of region names according to a list of region ids
    INPUT
        reg_ids is the list of region id
    OUTPUT
        reg_names is the list of region names
    '''
    reg_names = list()
    for reg_id in reg_ids:
        reg_names.append(get_reg_name(reg_id))
    return reg_names

list_dept_ids = ['45', 21, '13', '4']
list_reg_ids = [1, '84', 32, '93']

list_dept_names = get_dep_names(list_dept_ids)
print('list_dept_names:', list_dept_names)
list_reg_names = get_reg_names(list_reg_ids)
print('list_reg_names:', list_reg_names)

list_dept_names: ['Loiret', "Côte-d'Or", 'Bouches-du-Rhône', 'Alpes-de-Haute-Provence']
list_reg_names: ['Guadeloupe', 'Auvergne et Rhône-Alpes', 'Hauts-de-France', "Provence-Alpes-Côte d'Azur"]


In [127]:
# remove columns:
print(df.shape)
df.drop(columns=['reg_rea', 'reg_incid_rea', 'cv_dose1', 'R', 'lib_dep', 'lib_reg'], inplace=True)
print(df.shape)
df.head(5)

(101707, 22)
(101707, 16)


,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j
0,01,2020-03-18,84,NaN,NaN,0.062612,2,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN
1,01,2020-03-19,84,NaN,NaN,0.132379,2,0,1,0,1.0,0.0,0.0,0.0,NaN,NaN
2,01,2020-03-20,84,NaN,NaN,0.155635,2,0,1,0,0.0,0.0,1.0,0.0,NaN,NaN
3,01,2020-03-21,84,NaN,NaN,0.173524,4,0,1,0,3.0,0.0,0.0,0.0,NaN,NaN
4,01,2020-03-22,84,NaN,NaN,0.212880,8,1,2,0,3.0,1.0,1.0,0.0,NaN,NaN


In [128]:
# Finally I remove all rows containing at least one nan (loose 5.9% of the dataset)
df.dropna(axis='index', how='any', inplace=True)
print(df.shape)

(95748, 16)


In [129]:
display_columns_info(df)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
dep           	101		0.11 %		  object  		0	0.0 %
date          	948		0.99 %		  object  		0	0.0 %
analyse dates
reg           	18		0.02 %		  object  		0	0.0 %
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
rad           	16209		16.93 %		  int64  		0	0.0 %
dchosp        	4920		5.14 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_rad     	165		0.17 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %


In [130]:
def replace_invalid_data(df, label):
    '''
    DESCRIPtION
        Replace invalid data like nan or infinite by values
    INPUT
        df is the dataframe to work on
        label of the category (string) where invalid data will be replaced
    OUTPUT
        df is the dataframe modified
    '''
    # Replace nan values by 0
    df[label] = df[label].fillna(0)

    # Replace infinite (inf) values by the maximum of the serie
    maxi = df[label].max()
    df[label].replace([np.inf, -np.inf], maxi, inplace=True)  

    return df

In [131]:
display_columns_info(df)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
dep           	101		0.11 %		  object  		0	0.0 %
date          	948		0.99 %		  object  		0	0.0 %
analyse dates
reg           	18		0.02 %		  object  		0	0.0 %
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
rad           	16209		16.93 %		  int64  		0	0.0 %
dchosp        	4920		5.14 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_rad     	165		0.17 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %


In [132]:
# -----------------------------------------------------------------------------
# Try to get dummies on dep
# Since there is a lot, wait and see.

def dummy_data(df, cat):
    '''
    DESCRIPTION
        add dummy data to df removing the source of dummy
    INPUT
        df is the initial dataframe to work on
        cat is the name of the category we want dummying
    OUTPUT
        df_dummy is the dataframe as copy of df but with dummy columns,
         removing the source of dummy
    '''
    df_dummy = df.copy(deep=True)

    print('shape df:', df.shape)
    #print('df:', df.head())

    # Create a new dataframe for preparing dummies from a copy of the 
    #  selected category 
    df_cat = pd.DataFrame([])
    df_cat[cat] = np.array(df[cat])
    print('shape df_cat:', df_cat.shape)
    #print('df_cat:\n', df_cat.head())

    # Convert category values to just numbers 0 or 1 (dummy values) of category
    df_dum = pd.get_dummies(df_cat[cat], dtype=int)
    print('shape df_dum:', df_dum.shape)
    #print('df_dum:\n', df_dum.head())

    '''
    df_dummy = pd.concat([df, df_dum], axis=1, join='outer', ignore_index=False, sort=False)
    print('shape df_dummy:', df_dummy.shape)
    #print('df_dummy:\n', df_dummy.head())
    '''
    for col in df_dum.columns:
        #print('this ...', df_dum[col].shape)
        df_dummy[col] = np.array(df_dum[col])
        #print('into this ...', df_dummy.shape)
    print('shape df_dummy:', df_dummy.shape)

    # Join df_dum to df_cat; [cat] will make the link with df
    # otherwise, we have additional rows with nan
    #for col in df_dum.columns:
    #    df_cat[col] = np.array(df_dum[col])
    #print('shape df_cat:', df_cat.shape)

    # Remove invalid data
    #for col in categories.columns:
    #    categories['tx_test'] = categories['tx_test'].fillna(0)  # Replace nan values by 0
    #    categories['tx_test'].replace([np.inf, -np.inf], 0, inplace=True)  # Replace infinite value (inf) by 0
    #categories.fillna(value=0, inplace=True)  # Replace nan values by 0
    #categories.replace([np.inf, -np.inf], 0, inplace=True)  # Replace infinite value (inf) by 0

    # Replace category column in df with new category columns    
    # by concatenating the original dataframe with the new `categories` dataframe
    #df_dummy = pd.concat([df, categories], axis=1, join='outer')
    #df_dummy = pd.merge(df, df_cat, how='outer', on=[cat])
    #df_dummy = pd.merge(df, df_cat, on=[cat])
    '''
    df_dummy = df.copy(deep=True)
    print('shape df_dummy:', df_dummy.shape)

    for col in df_dum.columns:
        print('  + {} shape: {}'.format(col, df_dum[col].shape))
        df_dummy[col] = np.array(df_dum[col])
    print('shape df_dummy +:', df_dummy.shape)

    df_dummy = pd.merge(df_dummy, df_dum)


    #df_dummy = pd.concat([df_dummy, df_dum], axis=1)
    print('shape df_dummy concat:', df_dummy.shape)
    '''
    # Drop the original category column from `df_dummy`
    df_dummy.drop(columns=cat, inplace=True)
    print('shape df_dummy drop:', df_dummy.shape)

    # Remove duplicates
    # - check number of duplicates
    duplicates = df_dummy.duplicated().sum()
    if duplicates > 0:
        df_dummy.drop_duplicates(keep='first', inplace=True)
    print('shape df_dummy dupl:', df_dummy.shape)

    # Remove invalid data
    #categories.fillna(value=0, inplace=True)  # Replace nan values by 0
    #categories.replace([np.inf, -np.inf], 0, inplace=True)  # Replace infinite value (inf) by 0
    
    # clean df_dummy from rows with full nan values
    # otherwise, at least the row is a nana row.
    df_dummy.dropna(axis='index', how='all', inplace=True)
    print('shape df_dummy dropna:', df_dummy.shape)

    return df_dummy


In [133]:
# Add dummy values of 'dep'
# df = dummy_data(df, 'dep')
# df.head()

In [134]:
display_columns_info(df)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
dep           	101		0.11 %		  object  		0	0.0 %
date          	948		0.99 %		  object  		0	0.0 %
analyse dates
reg           	18		0.02 %		  object  		0	0.0 %
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
rad           	16209		16.93 %		  int64  		0	0.0 %
dchosp        	4920		5.14 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_rad     	165		0.17 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %


In [135]:
# Copy df for linear model
df_lin = df.copy(deep=True)
# remove the date (str) and dep (str)
if 'dep' in df_lin.columns:
    df_lin.drop(columns=['date', 'dep', 'reg', 'dchosp', 'rad', 'incid_rad'], inplace=True)  # without dummies
else:
    df_lin.drop(columns=['date', 'reg', 'dchosp', 'rad', 'incid_rad'], inplace=True)  # with dummies
print(df_lin.head())
print(df_lin.shape)

    tx_pos  tx_incid        TO  hosp  rea  incid_hosp  incid_rea  \
56    2.65      1.37  0.400716   139    8         6.0        0.0   
57    2.32      2.74  0.377460   137    8         4.0        0.0   
58    1.87      3.50  0.366726   135    7         4.0        0.0   
59    1.67      3.50  0.338104   134    6         1.0        0.0   
60    1.63      3.50  0.338104   133    6         1.0        0.0   

    incid_dchosp  pos  pos_7j  
56           0.0  9.0     9.0  
57           0.0  9.0    18.0  
58           1.0  5.0    23.0  
59           1.0  0.0    23.0  
60           0.0  0.0    23.0  
(95748, 10)


In [136]:
display_columns_info(df_lin)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %


In [137]:
df_lin.dropna(axis='index', how='any', inplace=True)
print(df_lin.shape)
print(df_lin.head())

(95748, 10)
    tx_pos  tx_incid        TO  hosp  rea  incid_hosp  incid_rea  \
56    2.65      1.37  0.400716   139    8         6.0        0.0   
57    2.32      2.74  0.377460   137    8         4.0        0.0   
58    1.87      3.50  0.366726   135    7         4.0        0.0   
59    1.67      3.50  0.338104   134    6         1.0        0.0   
60    1.63      3.50  0.338104   133    6         1.0        0.0   

    incid_dchosp  pos  pos_7j  
56           0.0  9.0     9.0  
57           0.0  9.0    18.0  
58           1.0  5.0    23.0  
59           1.0  0.0    23.0  
60           0.0  0.0    23.0  


In [138]:
display_columns_info(df_lin)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %


In [139]:
# Build a model
# - define the target
cat_response = 'pos_7j'
cat_variables = np.array(df_lin.columns) # useless, included into the get X/y function

In [140]:
# -----------------------------------------------------------------------------
def get_X_y(df: object, response:str):
    '''
    DESCRIPTION
        Split df into exploratory X data and response y data
    INPUT
           df      : pandas dataframe
           response : target variable
    OUTPUT
           X : A matrix holding all of the variables you want to consider
                when predicting the response
           y : the corresponding response vector
    '''
    # output
    X, y = None, None

    print(" - Spliting data into X/y ...")
    # Get the Response var
    if response in df.columns:
        # try:
        # Split into explanatory and response variables (1/2)
        #  Get response variable
        y = df[response]
        df = df.drop(columns=[response])  # Remove pred_name from df
        # except:
        #     print("    CAUTION: Unable to get the response data in df")
        #     y = None
    else:
        print(" - CAUTION: Unable to find the response in df")
        y = None

    # Get the Exploratory vars
    # try:
    # Split into explanatory and response variables (2/2)
    #  Get the input variables i.e. at this level just a copy of df
    X = df.copy(deep=True)
    # except:
    #     print("    CAUTION: Unable to get the exploratory variables (X)")
    #     X = None

    if X is None:
        print(' - No X found')
    if y is None:
        print(' - No y found')

    del df, response
    return X, y

# Split into Response y / Exploratory variables X
X, y = get_X_y(df_lin, cat_response)

 - Spliting data into X/y ...


In [141]:
# Modeling
# -----------------------------------------------------------------------------
def get_model(X: object, y: object, testrate=.3):
    '''
    DESCRIPTION
        Returns a linear prediction model according to train data,
        and return r2 scores on train and test data.
    INPUT
           X          : explanatory variables object
           y          : response variable object
           testrate   : proportion of the dataset to include in
                         the test split,between 0.0 and 1.0;
                         default value = 0.3
    OUTPUT
            model : linear regression model object from sklearn
            score : Merge of mean square error value between Train &
                     Test data set according to the proposed model
            list of X_train and y_train
            list of X_test and y_test
    '''
    model, score = None, 0
    Xtrain, Xtest, ytrain, ytest = None, None, None, None

    # sub-variables
    y_pred, msg, split = None, None, False
    acc_score_train, acc_score_test = None, None
    r2_score_train, r2_score_test = None, None
    mdl_score_train, mdl_score_test = None, None
    
    print(" - Spliting data into X/y ...")
    if (X is not None) and (y is not None):

        # Split into train and test X/y data set
        #  to establish the model and score it
        print("\tTraining ...")
        Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
                                                        test_size=testrate,
                                                        random_state=42)
        if (Xtrain is not None) and (Xtest is not None) and \
           (ytrain is not None) and (ytest is not None):
            split = True

        '''
        # Work on dtype
        recensed_type = {}
        for var in X.columns:
            tip = X[var].dtypes
            if tip not in recensed_type:
                recensed_type[tip] = 1
            else:
                recensed_type[tip] = recensed_type[tip] + 1
        '''

        # Establish model
        print("\tModeling ...")
        if split:

            # Linear model from scikit-learn:
            #  https://scikit-learn.org/stable/modules/linear_model.html#

            # Linear Regression
            model = LinearRegression()

            # Linear model Lasso
            # model = linear_model.Lasso(alpha=0.1)

            # Ridge regression and classification
            # model = linear_model.Ridge(alpha=.5)

            # BayesianRidge()
            # model = linear_model.BayesianRidge()

            # LogisticRegression
            # model = linear_model.LogisticRegression(random_state=0)
            # very long -> not achieve

            # generalized linear model TweedieRegressor
            # model = linear_model.TweedieRegressor(power=1, alpha=0.5, link='log')
            # Does not work!

            # Stochastic Gradient Descent
            # model = linear_model.SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
            # Does not provide good result

            # model  = linear_model.Perceptron(tol=1e-3, random_state=0)
            # Does not provide good result

            # fit the model
            model.fit(Xtrain, ytrain)

            # Test the model on test data
            y_pred = model.predict(Xtest)
        else:
            y_pred = None

        # Evaluate this model
        if y_pred is not None:
            # Get metrics with a model by Regression
            print("\tMetrics:")
            
            # Get the model's score
            # Return the coefficient of determination of the prediction
            mdl_score_train = model.score(Xtrain, ytrain)
            print("\t - Model score (train):", mdl_score_train)
            mdl_score_test = model.score(Xtest, ytest)
            print("\t - Model score (test):", mdl_score_test)

            # Accuracy_score (https://scikit-learn.org/stable/modules/
            # generated/sklearn.metrics.accuracy_score.html)
            # Confusion matrix (https://scikit-learn.org/stable/modules/
            # generated/sklearn.metrics.confusion_matrix.html)
            #  not appropriate for my purpose
            # Common pitfalls (https://scikit-learn.org/stable/common_
            #  pitfalls.html): mean_sqaured_error and r2_score

            # According to https://stackoverflow.com/questions/37367405/
            #  python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
            # ... Accuracy score is only for classification problems
            # acc_score_train = accuracy_score(y_train, model.predict(X_train))
            # acc_score_test = accuracy_score(y_test, model.predict(X_test))
            # if debug: print("    - acc_score_train:", acc_score_train)
            # if debug: print("    - acc_score_test:", acc_score_test)

            # always according to https://stackoverflow.com/questions/37367405/
            #  python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
            # For regression problems, use: R2 Score, MSE (Mean Squared Error),
            # RMSE (Root Mean Squared Error).
            r2_score_train = r2_score(ytrain, model.predict(Xtrain))
            r2_score_test = r2_score(ytest, model.predict(Xtest))
            print("\t - r2 score (train):", r2_score_train)
            print("\t - r2 score (test):", r2_score_test)

            score = r2_score_train
        else:
            print("\t - Model not found")

    del X, y, testrate, y_pred, acc_score_train, acc_score_test, split
    del r2_score_train, r2_score_test, mdl_score_train, mdl_score_test
    return model, score, [Xtrain, ytrain], [Xtest, ytest]


# modeling
model, score, Xy_train, Xy_test = get_model(X, y)


 - Spliting data into X/y ...
	Training ...
	Modeling ...
	Metrics:
	 - Model score (train): 0.881512098436752
	 - Model score (test): 0.8738613401903677
	 - r2 score (train): 0.881512098436752
	 - r2 score (test): 0.8738613401903677


In [142]:
def coef_weights(model, X_train) -> object:
    '''
    returns a dataframe with coefficients of the model
     (real and absolute values) sorted in the descending order
     of the absolute values

    input:
           model     : model for which we are looking coefficients
           X_train   : the training data
    output:
            coefs_df : dataframe with model's coefficients; that can be
                        used to understand the most influential coefficients
                        in a linear model by providing the coefficient
                        estimates along with the name of the variable
                        attached to the coefficient.
    '''
    # function: get model's coefficients

    coefs_df = pd.DataFrame()
    # Get name of every column in front  of its coefficients
    coefs_df['est_int'] = X_train.columns
    # get coefficients of the linear model
    coefs_df['coefs'] = model.coef_
    # get absolute value of these coefficients
    coefs_df['abs_coefs'] = np.abs(model.coef_)
    # Sort coefficient by descending order
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)

    del model, X_train
    return coefs_df


print('  > Define impact of categories on the model:')
coef_df = None

# Compute coefficient of weight on every category for this model
coef_df = coef_weights(model, Xy_train[0])  # Xy_train = [Xtrain, ytrain]
pd.set_option("display.max_rows", None, "display.max_columns", None)
print('\t', coef_df)



  > Define impact of categories on the model:
	         est_int       coefs   abs_coefs
2            TO -966.678578  966.678578
7  incid_dchosp -167.490157  167.490157
6     incid_rea  -75.851722   75.851722
0        tx_pos  -28.293308   28.293308
5    incid_hosp   10.533020   10.533020
4           rea    9.841827    9.841827
3          hosp    4.663104    4.663104
8           pos    4.059020    4.059020
1      tx_incid    3.190276    3.190276


In [143]:
# Question 1
# Create a column with the result of 'tx_pos' / 'tx_incid' = tx_test
df['tx_test'] = df['tx_pos'].div(df['tx_incid'])  # div function allows supporting 
                                                  #  missing value (in case we miss
                                                  #  something) with fill_value
# Replace invalid data
df = replace_invalid_data(df, 'tx_test')
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714


In [144]:
display_columns_info(df)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
dep           	101		0.11 %		  object  		0	0.0 %
date          	948		0.99 %		  object  		0	0.0 %
analyse dates
reg           	18		0.02 %		  object  		0	0.0 %
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
rad           	16209		16.93 %		  int64  		0	0.0 %
dchosp        	4920		5.14 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_rad     	165		0.17 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %
tx_test       	91399		95.46 %		  float64  		0	0.0 %


In [145]:
# Question 2 - monitor the trend of hospitals occupancy per department
# Create a column with the result of ('incid_hosp' - 'incid_rad' - 'incid_dchosp')
#  / ('hosp' + 'rea') = 'incid_IO'
incid_hosp = np.array(df['incid_hosp'])
incid_rad = np.array(df['incid_rad'])
incid_dchosp = np.array(df['incid_dchosp'])
hosp = np.array(df['hosp'])
rea = np.array(df['rea'])

# Avoid division by zero
people_hospitalized = np.add(hosp, rea)

# Replace 0 by 1 people to allow division 
ones = np.ones(np.shape(people_hospitalized), dtype=type(people_hospitalized))
people_hospitalized = np.where(np.absolute(people_hospitalized) < 1, ones, people_hospitalized)

tx_incid_IO = np.divide(np.subtract(incid_hosp, np.add(incid_rad, incid_dchosp)), people_hospitalized)
df['tx_incid_IO'] = tx_incid_IO

df['tx_incid_IO'] = df['tx_incid_IO'].fillna(0)  # Replace nan values by 0
df['tx_incid_IO'].replace([np.inf, -np.inf], 0, inplace=True)  #Replace infinite value (inf) by 0
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test,tx_incid_IO
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307,0.013605
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715,0.000000
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286,-0.014085
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143,-0.014286
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714,0.000000


In [146]:
display_columns_info(df)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
dep           	101		0.11 %		  object  		0	0.0 %
date          	948		0.99 %		  object  		0	0.0 %
analyse dates
reg           	18		0.02 %		  object  		0	0.0 %
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
rad           	16209		16.93 %		  int64  		0	0.0 %
dchosp        	4920		5.14 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_rad     	165		0.17 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %
tx_test       	91399		95.46 %		  float64  		0	0.0 %
tx_incid_IO   	13647		14.25 %		  float64  		0	0.0 %


In [147]:
# Question 3-1
# Compute Rate of people admitted in intensive care over people hospitalized
df['tx_rea'] = df['incid_rea'].div(df['incid_hosp'])
# Replace invalid data
df = replace_invalid_data(df, 'tx_rea')
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test,tx_incid_IO,tx_rea
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307,0.013605,0.0
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715,0.000000,0.0
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286,-0.014085,0.0
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143,-0.014286,0.0
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714,0.000000,0.0


In [148]:
display_columns_info(df)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
dep           	101		0.11 %		  object  		0	0.0 %
date          	948		0.99 %		  object  		0	0.0 %
analyse dates
reg           	18		0.02 %		  object  		0	0.0 %
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
rad           	16209		16.93 %		  int64  		0	0.0 %
dchosp        	4920		5.14 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_rad     	165		0.17 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %
tx_test       	91399		95.46 %		  float64  		0	0.0 %
tx_incid_IO   	13647		14.25 %		  float64  		0	0.0 %
tx_rea        	1250		1.31 %	

In [149]:
# Question 3-2
# Compute Rate of deceases at hospital over people admitted in intensive care
df['tx_dchosp'] = df['incid_dchosp'].div(df['incid_rea'], fill_value=0.0)
# Replace invalid data
df = replace_invalid_data(df, 'tx_dchosp')
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test,tx_incid_IO,tx_rea,tx_dchosp
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307,0.013605,0.0,0.0
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715,0.000000,0.0,0.0
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286,-0.014085,0.0,inf
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143,-0.014286,0.0,inf
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714,0.000000,0.0,0.0


In [150]:
display_columns_info(df)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
dep           	101		0.11 %		  object  		0	0.0 %
date          	948		0.99 %		  object  		0	0.0 %
analyse dates
reg           	18		0.02 %		  object  		0	0.0 %
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
rad           	16209		16.93 %		  int64  		0	0.0 %
dchosp        	4920		5.14 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_rad     	165		0.17 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %
tx_test       	91399		95.46 %		  float64  		0	0.0 %
tx_incid_IO   	13647		14.25 %		  float64  		0	0.0 %
tx_rea        	1250		1.31 %	

In [151]:
# -----------------------------------------------------------------------------
# Get a list that gives the nb of past days from beginning of the year
#  for every month
day_by_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
cnt_day_at_start_month = []
cnt = 0
for i in day_by_month:
    cnt += i 
    cnt_day_at_start_month.append(cnt)
calendar = cnt_day_at_start_month
print('calender:', calendar)

calender: [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]


In [152]:
def get_cnt_day_at_start_month(m, calendar):
    '''
    DESCRIPTION
        get the number of past days from the beginning of year and at 
         beginning of the selected month.
    INPUT
        m is the selected month (str of int, int or float)
    OUTPUT
        cnt is the count of days (int) from beginning of the year at start of
         the selected month.
    '''
    cnt = calendar[int(m)-1]
    return cnt
m_test = get_cnt_day_at_start_month(3, calendar)
print('for month 3-March, i obtain {} vs {} expected'.format(m_test, 59))

for month 3-March, i obtain 59 vs 59 expected


In [153]:
def get_timestamp_array(dates_str):
    '''
    DESCRIPTION
        Convert an np.array of dates in string format
        to a timestamp in number of days.
    INPUT
        dates_str is a np.array of dates YYYY-MM-DD in string format
    OUTPUT
        timestamp is np.array with data converted in number of days (int)
    '''
    timestamp = []
    j = 0
    for d in dates_str:
        try:
            y = float(d[:4])
            m = float(d[5:7])
            d = float(d[8:10])
            # convert the data in number of days
            cnt_days_for_months = float(get_cnt_day_at_start_month(m, calendar))
            ts = y * 365.24219 + cnt_days_for_months + d
        except:
            ts = 0.0
        ts_r = round(ts, 0)
        timestamp.append(int(ts_r))
        j += 1
    return np.array(timestamp)

test_dates = np.array(['2022-01-01', '2022-12-31'])
timestamp_arr = get_timestamp_array(test_dates)
print('expected: {} vs computed {}'.format([738521, 73885], timestamp_arr))

expected: [738521, 73885] vs computed [738521 738885]


In [154]:
# Get the timestamp column in df
df['timestamp'] = get_timestamp_array(np.array(df['date']))
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test,tx_incid_IO,tx_rea,tx_dchosp,timestamp
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307,0.013605,0.0,0.0,737922
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715,0.000000,0.0,0.0,737923
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286,-0.014085,0.0,inf,737924
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143,-0.014286,0.0,inf,737925
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714,0.000000,0.0,0.0,737926


In [155]:
display_columns_info(df)

column label	nb of unique	rate of unique	  type		nb of nan	rate of nan
dep           	101		0.11 %		  object  		0	0.0 %
date          	948		0.99 %		  object  		0	0.0 %
analyse dates
reg           	18		0.02 %		  object  		0	0.0 %
tx_pos        	4581		4.78 %		  float64  		0	0.0 %
tx_incid      	50497		52.74 %		  float64  		0	0.0 %
TO            	3548		3.71 %		  float64  		0	0.0 %
hosp          	1548		1.62 %		  int64  		0	0.0 %
rea           	402		0.42 %		  int64  		0	0.0 %
rad           	16209		16.93 %		  int64  		0	0.0 %
dchosp        	4920		5.14 %		  int64  		0	0.0 %
incid_hosp    	203		0.21 %		  float64  		0	0.0 %
incid_rea     	57		0.06 %		  float64  		0	0.0 %
incid_rad     	165		0.17 %		  float64  		0	0.0 %
incid_dchosp  	41		0.04 %		  float64  		0	0.0 %
pos           	4331		4.52 %		  float64  		0	0.0 %
pos_7j        	13405		14.0 %		  float64  		0	0.0 %
tx_test       	91399		95.46 %		  float64  		0	0.0 %
tx_incid_IO   	13647		14.25 %		  float64  		0	0.0 %
tx_rea        	1250		1.31 %	

In [156]:
lst = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
sampling_rate = 7

ref_id_grid = list()
maxi = 1 + int(round(len(lst) / sampling_rate, 0))
for x in range(maxi):
    ref_id_grid.append(x * sampling_rate)
print(ref_id_grid)

[0, 7, 14]


In [157]:
if 'dep' in df.columns:
    df.groupby(['dep', 'timestamp'])  # If no dummy applied
else:
    df.groupby(['timestamp'])  # if dummy applied
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test,tx_incid_IO,tx_rea,tx_dchosp,timestamp
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307,0.013605,0.0,0.0,737922
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715,0.000000,0.0,0.0,737923
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286,-0.014085,0.0,inf,737924
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143,-0.014286,0.0,inf,737925
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714,0.000000,0.0,0.0,737926


In [158]:
# -----------------------------------------------------------------------------
deps = pd.unique(df['dep'])
print('deps:\n', deps)

deps:
 ['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '2A' '2B' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41'
 '42' '43' '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55'
 '56' '57' '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69'
 '70' '71' '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83'
 '84' '85' '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '971' '972'
 '973' '974' '976']


In [159]:
# -----------------------------------------------------------------------------
def get_values_at_last_date(df, category, deps):
    '''
    DESCRIPTION
        Provide values of the selected category at the last date available in
         the dataframe, for every department.
    INPUT
        category is the name of the columns for which we want values at the 
         last date for every department
    OUPUT
        values is a dictionary with {dep: {'date': date, 'value': value of the
         category at the stored date}}
    '''

    values = dict()

    # format a dedicated list of data for the search
    lst = [np.array(df['dep']),
        np.array(df['date']),
        np.array(df['timestamp']),
        np.array(df[category])]

    # Run over every department
    for dep in deps:
        time_ts = 0
        id_max_date = 0
        # run along the rows of the dataframe
        for i, j in enumerate(lst[0]):
            # When I met the selected department
            #  and the timestamp is higher (ensure get finally the most
            #  recent timestamp/date and related values of the category),
            #  we stored date and value of the category in values, relatively
            #  to the department. 
            if (j == dep) and (time_ts < lst[2][i]):
                id_max_date = i
                time_ts = lst[2][i]

        values[dep] = {'date': lst[1][i], 'value': lst[3][i]}
    return values

# test the function
test_tx = get_values_at_last_date(df, 'tx_test', deps)
print('result in dep 01:', test_tx['01'])


result in dep 01: {'date': '2022-12-16', 'value': 0.11546685673556666}


In [160]:
# -----------------------------------------------------------------------------
# Convert result for preparing vizualization
# For vizualization, I need to get values like
# date, {dep: values, ...}

def format_values_for_vizu(dico_values, nb_decimal):
    '''
    DESCRIPTION
        format values for a vizualization on a map
    INPUT
        dico_values is a dictionary such {dep: {'date': date, 'value': value
         of the category at the stored date}}
    OUTPUT
        series is a dictionary such as {date1: {depY: valueY, ...}, date2: {...}}
    '''

    series = dict()

    dates = list()
    for dep, j in dico_values.items():
        date = j['date']
        if date not in dates: # Create a sub-dictionary at every new date
            dates.append(date)
            series[date] = dict()
        series[date][dep] = round(j['value'], nb_decimal)

    return series


In [161]:
# -----------------------------------------------------------------------------
# Question 1: visualization
# format for vizualization
tx_test = get_values_at_last_date(df, 'tx_test', deps)
# Get the selected data
tx_test_vizu = format_values_for_vizu(tx_test, 3)
# Data for visualization
print('tx_test_vizu:\n', tx_test_vizu)


tx_test_vizu:
 {'2022-12-16': {'01': 0.115, '02': 0.115, '03': 0.115, '04': 0.115, '05': 0.115, '06': 0.115, '07': 0.115, '08': 0.115, '09': 0.115, '10': 0.115, '11': 0.115, '12': 0.115, '13': 0.115, '14': 0.115, '15': 0.115, '16': 0.115, '17': 0.115, '18': 0.115, '19': 0.115, '21': 0.115, '22': 0.115, '23': 0.115, '24': 0.115, '25': 0.115, '26': 0.115, '27': 0.115, '28': 0.115, '29': 0.115, '2A': 0.115, '2B': 0.115, '30': 0.115, '31': 0.115, '32': 0.115, '33': 0.115, '34': 0.115, '35': 0.115, '36': 0.115, '37': 0.115, '38': 0.115, '39': 0.115, '40': 0.115, '41': 0.115, '42': 0.115, '43': 0.115, '44': 0.115, '45': 0.115, '46': 0.115, '47': 0.115, '48': 0.115, '49': 0.115, '50': 0.115, '51': 0.115, '52': 0.115, '53': 0.115, '54': 0.115, '55': 0.115, '56': 0.115, '57': 0.115, '58': 0.115, '59': 0.115, '60': 0.115, '61': 0.115, '62': 0.115, '63': 0.115, '64': 0.115, '65': 0.115, '66': 0.115, '67': 0.115, '68': 0.115, '69': 0.115, '70': 0.115, '71': 0.115, '72': 0.115, '73': 0.115, '74': 0

In [162]:
# -----------------------------------------------------------------------------
# Question 2: visualization
# Get the selected values
tx_incid_IO = get_values_at_last_date(df, 'tx_incid_IO', deps)
# format for vizualization
tx_IO_vizu = format_values_for_vizu(tx_incid_IO, 3)
# Data for visualization
print("tx_IO_vizu:\n", tx_IO_vizu)


tx_IO_vizu:
 {'2022-12-16': {'01': 0.0, '02': 0.0, '03': 0.0, '04': 0.0, '05': 0.0, '06': 0.0, '07': 0.0, '08': 0.0, '09': 0.0, '10': 0.0, '11': 0.0, '12': 0.0, '13': 0.0, '14': 0.0, '15': 0.0, '16': 0.0, '17': 0.0, '18': 0.0, '19': 0.0, '21': 0.0, '22': 0.0, '23': 0.0, '24': 0.0, '25': 0.0, '26': 0.0, '27': 0.0, '28': 0.0, '29': 0.0, '2A': 0.0, '2B': 0.0, '30': 0.0, '31': 0.0, '32': 0.0, '33': 0.0, '34': 0.0, '35': 0.0, '36': 0.0, '37': 0.0, '38': 0.0, '39': 0.0, '40': 0.0, '41': 0.0, '42': 0.0, '43': 0.0, '44': 0.0, '45': 0.0, '46': 0.0, '47': 0.0, '48': 0.0, '49': 0.0, '50': 0.0, '51': 0.0, '52': 0.0, '53': 0.0, '54': 0.0, '55': 0.0, '56': 0.0, '57': 0.0, '58': 0.0, '59': 0.0, '60': 0.0, '61': 0.0, '62': 0.0, '63': 0.0, '64': 0.0, '65': 0.0, '66': 0.0, '67': 0.0, '68': 0.0, '69': 0.0, '70': 0.0, '71': 0.0, '72': 0.0, '73': 0.0, '74': 0.0, '75': 0.0, '76': 0.0, '77': 0.0, '78': 0.0, '79': 0.0, '80': 0.0, '81': 0.0, '82': 0.0, '83': 0.0, '84': 0.0, '85': 0.0, '86': 0.0, '87': 0.0, '88

In [163]:
# -----------------------------------------------------------------------------
# Question 3-1: visualization
# Get the selected values
tx_rea = get_values_at_last_date(df, 'tx_rea', deps)
# format for vizualization
tx_rea_vizu = format_values_for_vizu(tx_rea, 3)
# Data for visualization
print("tx_rea_vizu:\n", tx_rea_vizu)

tx_rea_vizu:
 {'2022-12-16': {'01': 0.0, '02': 0.0, '03': 0.0, '04': 0.0, '05': 0.0, '06': 0.0, '07': 0.0, '08': 0.0, '09': 0.0, '10': 0.0, '11': 0.0, '12': 0.0, '13': 0.0, '14': 0.0, '15': 0.0, '16': 0.0, '17': 0.0, '18': 0.0, '19': 0.0, '21': 0.0, '22': 0.0, '23': 0.0, '24': 0.0, '25': 0.0, '26': 0.0, '27': 0.0, '28': 0.0, '29': 0.0, '2A': 0.0, '2B': 0.0, '30': 0.0, '31': 0.0, '32': 0.0, '33': 0.0, '34': 0.0, '35': 0.0, '36': 0.0, '37': 0.0, '38': 0.0, '39': 0.0, '40': 0.0, '41': 0.0, '42': 0.0, '43': 0.0, '44': 0.0, '45': 0.0, '46': 0.0, '47': 0.0, '48': 0.0, '49': 0.0, '50': 0.0, '51': 0.0, '52': 0.0, '53': 0.0, '54': 0.0, '55': 0.0, '56': 0.0, '57': 0.0, '58': 0.0, '59': 0.0, '60': 0.0, '61': 0.0, '62': 0.0, '63': 0.0, '64': 0.0, '65': 0.0, '66': 0.0, '67': 0.0, '68': 0.0, '69': 0.0, '70': 0.0, '71': 0.0, '72': 0.0, '73': 0.0, '74': 0.0, '75': 0.0, '76': 0.0, '77': 0.0, '78': 0.0, '79': 0.0, '80': 0.0, '81': 0.0, '82': 0.0, '83': 0.0, '84': 0.0, '85': 0.0, '86': 0.0, '87': 0.0, '8

In [164]:
# Question 4: preparation for visualization
# I need to get all values of a categories along the time for a selected department

def get_timeserie(category, dep, duration_week):
    
    # get dataframe the selecte category with time and departments
    df_ = df[['dep', 'date', 'timestamp', category]]

    # keep only rows with the selected dep
    df_ = df_[df_['dep'] == dep]

    # sort dataframe by descending timestamp (most recent at first)
    df_.sort_values(by=['timestamp'], ascending=False, inplace=True)
    
    # get last timestamp (most recent date)
    timestamp_max = df_['timestamp'].values[0]

    # compute start of the time window
    timestamp_min = timestamp_max - 7*duration_week  # unit here is the day
                                                     # 7 days per week

    # keep only dataframe on the required time stamp period
    df_ = df_[df_['timestamp'] > timestamp_min]

    # get expected values and related dates
    dates = df_['date'].values.tolist()
    values = df_[category].values.tolist()
    # revers is for getting time flow from left to right

    return list(reversed(dates)), list(reversed(values))

dates_hosp, values_hosp = get_timeserie('hosp', '31', 6*4)
dates_intensive, values_intensive = get_timeserie('rea', '31', 6*4)

In [165]:
# Questions 4: visualization
# Data for visualization
print("dates_hosp:\n", dates_hosp)
print("values_hosp:\n", values_hosp)
print("values_intensive:\n", values_intensive)

dates_hosp:
 ['2022-07-02', '2022-07-03', '2022-07-04', '2022-07-05', '2022-07-06', '2022-07-07', '2022-07-08', '2022-07-09', '2022-07-10', '2022-07-11', '2022-07-12', '2022-07-13', '2022-07-14', '2022-07-15', '2022-07-16', '2022-07-17', '2022-07-18', '2022-07-19', '2022-07-20', '2022-07-21', '2022-07-22', '2022-07-23', '2022-07-24', '2022-07-25', '2022-07-26', '2022-07-27', '2022-07-28', '2022-07-29', '2022-07-30', '2022-07-31', '2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04', '2022-08-05', '2022-08-06', '2022-08-07', '2022-08-08', '2022-08-09', '2022-08-10', '2022-08-11', '2022-08-12', '2022-08-13', '2022-08-14', '2022-08-15', '2022-08-16', '2022-08-17', '2022-08-18', '2022-08-19', '2022-08-20', '2022-08-21', '2022-08-22', '2022-08-23', '2022-08-24', '2022-08-25', '2022-08-26', '2022-08-27', '2022-08-28', '2022-08-29', '2022-08-30', '2022-08-31', '2022-09-01', '2022-09-02', '2022-09-03', '2022-09-04', '2022-09-05', '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09', '2022-